## NLP Assignment: RAGs For Open Domain Complex QA


In [40]:
from dexter.data.loaders.RetrieverDataset import RetrieverDataset
from dexter.retriever.dense.Contriever import Contriever
from dexter.config.constants import Split
from dexter.utils.metrics.retrieval.RetrievalMetrics import RetrievalMetrics
from dexter.utils.metrics.SimilarityMatch import CosineSimilarity as CosScore
from dexter.utils.metrics.CoverExactMatch import CoverExactMatch
from dexter.data.datastructures.hyperparameters.dpr import DenseHyperParams
import json
import torch
import random

In [41]:
# Check torch version and make sure cuda is enabled and available
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

2.5.1
True
12.4


In [42]:
# Since china doesn't have access to huggingface, I have manually downloaded the model, feel free to comment this.
from transformers import AutoTokenizer, AutoModel
model_path = '../contriever'
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModel.from_pretrained(model_path, local_files_only=True)

#model_path = "facebook/contriever" <- but uncomment this

### Task 1
Use off the shelf retriever (contriever) and extract contexts for each query to be given as input to a generative model. Use Exact Match or cover Exact Match as metric for evaluating generated answers. Experiment with k=1,3,5 for retrieving top-k contexts and report the performance on generating answers.

In [44]:
config_instance = DenseHyperParams(query_encoder_path=model_path,
                                    document_encoder_path=model_path
                                    ,batch_size=32)

loader = RetrieverDataset("wikimultihopqa","wiki_musique_corpus","config.ini",Split.DEV,tokenizer=None)
queries, qrels, corpus = loader.qrels()
con = Contriever(config_instance)

## wikimultihop <- it had this in the google colab notebook, should we use it?
# with open("data/wiki_musique_corpus.json") as f:
#     corpus = json.load(f)

Transforming passage dataset: 100%|██████████| 563424/563424 [00:01<00:00, 461275.40it/s]


Harley-Davidson Harley-Davidson
KeysView(<Section: Data-Path>)
12576


100%|██████████| 1200/1200 [01:32<00:00, 12.94it/s]


In [45]:
similarity_measure = CosScore()
k_values = [1,2,3]
metrics = RetrievalMetrics(k_values=k_values)

# Retrieve top-k contexts and evaluate with Exact Match
def evaluate_with_top_k(queries, corpus, k_values):
    response = con.retrieve(corpus, queries, top_k=max(k_values), score_function=similarity_measure)
    print("Indices retrieved:", len(response))
    print(metrics.evaluate_retrieval(qrels=qrels, results=response))

evaluate_with_top_k(queries, corpus, k_values)

token_emb torch.Size([1200, 35, 768])
sentence_emb torch.Size([1200, 768])


  0%|          | 32/563424 [00:00<45:33, 206.13it/s]

Starting encoding of contexts....


100%|██████████| 563424/563424 [26:18<00:00, 356.87it/s]


context_embeddings torch.Size([563424, 768])
Indices retrieved: 1200
({'NDCG@1': 0.425, 'NDCG@2': 0.38567, 'NDCG@3': 0.33839}, {'MAP@1': 0.04277, 'MAP@2': 0.06457, 'MAP@3': 0.07541}, {'Recall@1': 0.04277, 'Recall@2': 0.07528, 'Recall@3': 0.09379}, {'P@1': 0.425, 'P@2': 0.37417, 'P@3': 0.31083})


### Task 2
Repeat the above experiment without the retriever, using only oracle contexts as input. Oracle
contexts are annotated documents provided for each question in dev.json.

### Task 3
Now randomly sample documents from the collection that are not relevant to the current query during inference on the evaluation set. Combine these documents with the top-k relevant documents and use them as input to the LLM for answering a query. You can decide the ratios to mix the relevant and the random documents that serve as noise. Analyze the performance.

### Task 4
 In this step, we will adopt a more principled approach to sample negative documents to be used as input to the RAG setup. Using a retrieval model, sample hard negatives from the collection for the
current query instead of random documents to inject as noise. hard negatives are documents that are related and close to the query in the vector space but do not help answer the question. This can be sampled by retrieving documents not in the list of ground truth documents for a query as measure by dot product.

### Task 5
Train a retrieval model using ADORE [14]. ADORE is optimized with hard negatives in a dense retrieval setup. Hence, it may be able to discern more relevant documents from large collections and lead to improved downstream answer generation performance. Using this retriever, retrieve relevant contexts followed by answer generation using LLMs. Compare it to the baseline performance of contriever based LLM QA mentioned in step 2 above.